In [2]:
import os
spark_version = 'spark-3.1.3'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [3]:
# Connect to Postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-09-07 20:51:22--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.54MB/s    in 0.2s    

2022-09-07 20:51:22 (4.54 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [14]:
# Create connection
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Baby").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [15]:
# Load in data
from pyspark import SparkFiles
url = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Baby_v1_00.tsv.gz'
spark.sparkContext.addFile(src)
baby = spark.read.csv(SparkFiles.get('amazon_reviews_us_Baby_v1_00.tsv.gz'), sep='\t', timestampFormat='yyyy/MM/dd', inferSchema=True, header=True)
baby.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    9970739| R8EWA1OFT84NX|B00GSP5D94|     329991347|Summer Infant Swa...|            Baby|          5|            0|          0|   N|                Y|Great swaddled bl...|Loved these swadd...| 2015-08-31|
|         US|   23538442|R2JWY4YRQD4FOP|B00YYDDZGU|     646108902|Pacifier Clip Gir...|            Baby|          5|    

In [24]:
# Number of rows
rows = baby.count()
print(f'Total rows: {rows}')


Total rows: 1752932


In [49]:
#Transforming DataFrame to fit table schema 
review_id_table = baby.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_table.show(10)
review_id_table.printSchema()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R8EWA1OFT84NX|    9970739|B00GSP5D94|     329991347| 2015-08-31|
|R2JWY4YRQD4FOP|   23538442|B00YYDDZGU|     646108902| 2015-08-31|
| RL5ESX231LZ0B|    8273344|B00BUBNZC8|     642922361| 2015-08-31|
| RRMS9ZWJ2KD08|   24557753|B00AWLZFTS|     494272733| 2015-08-31|
|R14I3ZG5E6S7YM|   46263340|B00KM60D3Q|     305813185| 2015-08-31|
|R13EPSFP5DODN5|   24557753|B00PQMRZG4|     607341708| 2015-08-31|
| R6RBP4HTE67SY|   33520065|B005DL5970|     971881542| 2015-08-31|
|R15B3EU40RSU2W|   20241560|B00C6D2WL4|      93827401| 2015-08-31|
| RP4DD53A4ZJA2|    9987983|B0083973FK|     958629336| 2015-08-31|
|R2C99DJEO4RZ4K|   52570308|B00RLYG2S2|     147324304| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows

root
 |-- review_id: string (nullabl

In [50]:
#Transforming DataFrame to fit products schema 
products = baby[['product_id', 'product_title']]
products.dropDuplicates(['product_id'])
products.dropna()
products.show(10)
products.printSchema()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00GSP5D94|Summer Infant Swa...|
|B00YYDDZGU|Pacifier Clip Gir...|
|B00BUBNZC8|Udder Covers - Br...|
|B00AWLZFTS|Gerber Graduates ...|
|B00KM60D3Q|Summer Infant Ult...|
|B00PQMRZG4|Summer Infant Kee...|
|B005DL5970|Natural HE Powder...|
|B00C6D2WL4|Dr. Brown's Bottl...|
|B0083973FK|Sposie Booster Pa...|
|B00RLYG2S2|Abiie Beyond Wood...|
+----------+--------------------+
only showing top 10 rows

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [51]:
#Transforming DataFrame to fit customers schema 
customers = baby.select(["customer_id"])
customers = customers.groupBy("customer_id").count()
customers.orderBy("customer_id").select(["customer_id", "count"])
customers.show(10)
customers.printSchema()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   23042837|    2|
|   44524872|    1|
|   16711087|    1|
|   14116630|    3|
|   10548283|    1|
|   12879980|   15|
|   11913481|    1|
|   49875798|    3|
|   40993003|    1|
|    6272214|    3|
+-----------+-----+
only showing top 10 rows

root
 |-- customer_id: integer (nullable = true)
 |-- count: long (nullable = false)



In [46]:
#Transforming DataFrame to fit vine schema 
from pyspark.sql.types import * 
vine = baby.select(["review_id","star_rating", "helpful_votes", "total_votes", "vine"])
vine = vine.dropDuplicates(["review_id"])

vine = vine.withColumn("star_rating",vine["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine["total_votes"].cast(IntegerType()))

vine.show(10)
vine.printSchema()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100GNJU76M4Q9|          4|            0|          0|   N|
|R100RNZZU8P949|          5|            2|          2|   N|
|R102LHBUMFLRSZ|          3|            5|          6|   N|
|R102NVF0W1Y2Q1|          5|            5|          8|   N|
|R1030RGR05XVMJ|          5|            0|          0|   N|
|R1032V7632ZYV8|          5|            0|          0|   N|
|R1035LB8SN4J8C|          5|            0|          0|   N|
|R10459MPUEC8XH|          5|            0|          0|   N|
|R104RESTVELARK|          5|            5|          5|   N|
|R105Z2BW17EOMD|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true

In [55]:
#Push to AWS
mode = "append"
mode = 'append'
jdbc_url= 'jdbc:postgresql://<endpoint>:5432/<database>'
config = {'user':'<username>', 
          'password': '<password>', 
          'driver':'org.postgresql.Driver'}

In [58]:
#review_id_table.write.jdbc(url=jdbc_url, table="review_id_table", mode=mode, properties=config)
#products.write.jdbc(url=jdbc_url, table="products", mode=mode, properties=config)
#customers.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)
#vine.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)